In [20]:
import pandas as pd

In [ ]:
# used modified parser from here: https://github.com/rathishg/drugbank_ca/blob/master/drug_bank_xml2db.py

In [17]:
# read, parse, process, and write drugbank files

!python3 drugbank_xml_2_db.py -i data/drugbank/full_database.xml

Reading and parsing xml file.
Processing records.
Writing output files.
Done:
	drug_target: 19028 records
	drug_target_action: 212473 records
	drugs: 14594 records
	targets: 19028 records


In [30]:
# convert txt files to pandas dfs

drug_target = pd.read_csv("data/drugbank/full_database.xml.drug_target.txt", sep='\t')
drug_target_action = pd.read_csv("data/drugbank/full_database.xml.drug_target_action.txt", sep="\t")
drugs = pd.read_csv("data/drugbank/full_database.xml.drugs.txt", sep="\t")
targets = pd.read_csv("data/drugbank/full_database.xml.targets.txt", sep='\t')


In [126]:
# format data by lower casing all drug names

drugs['drug_name'] = [d.lower() for d in drugs['drug_name']]

heart_drugs = pd.read_csv("data/drugs_with_heart_se.csv")
heart_drugs['drug_name'] = [d.lower() for d in heart_drugs['drug_name']]

drug_name_target_name = pd.merge(pd.merge(drug_target_action, drugs, on = 'drug_id', how = 'left')[['target_id', 'drug_name', 'restrictions', 'action']], targets, on = 'target_id', how='left')[['drug_name', 'target_name', 'restrictions', 'action']]

In [127]:
# create db of all drugs that give heart-related side effects and their targets

heart_drugs_db = drug_name_target_name[drug_name_target_name['drug_name'].isin(list(heart_drugs.drug_name))]

In [128]:
# num unique drugs that had target info

print("num unique drugs with target info: " + str(len(heart_drugs_db.drug_name.unique())))

num unique drugs with target info: 664


In [129]:
# num unique targets 

heart_drug_targets = heart_drugs_db.target_name.unique()
print("num unique targets: " + str(len(heart_drug_targets)))

num unique targets: 773


In [130]:
# save heart - drug - target df

heart_drugs_db.to_csv("data/heart_drugs_targets.csv", index=None)

In [131]:
heart_drugs_db

,drug_name,target_name,restrictions,action
33,bivalirudin,Prothrombin,"approved,investigational",inhibitor
85,goserelin,Lutropin-choriogonadotropic hormone receptor,approved,agonist
86,goserelin,Gonadotropin-releasing hormone receptor,approved,agonist
259,desmopressin,Vasopressin V2 receptor,approved,agonist
260,desmopressin,Vasopressin V1a receptor,approved,agonist
...,...,...,...,...
17925,lormetazepam,GABA(A) Receptor,approved,ligand
17926,lormetazepam,GABA(A) Receptor Benzodiazepine Binding Site,approved,positive allosteric modulator
17927,lormetazepam,GABA(A) Receptor Benzodiazepine Binding Site,approved,ligand
17966,candesartan,Type-1 angiotensin II receptor,experimental,antagonist
